In [4]:
import vk_api

Предварительный анализ скачанных жсон

In [5]:
jposts = vk_api.load_json('posts.json')

In [6]:
jposts[:5]

[{'attachments': [{'photo': {'access_key': '008739e87d330314d1',
     'album_id': -7,
     'date': 1493664165,
     'height': 750,
     'id': 456239019,
     'owner_id': 23704358,
     'photo_130': 'https://pp.userapi.com/c638720/v638720358/3224b/TSsh8ZWB2oA.jpg',
     'photo_604': 'https://pp.userapi.com/c638720/v638720358/3224c/ZUKVH-KDb30.jpg',
     'photo_75': 'https://pp.userapi.com/c638720/v638720358/3224a/A--QVaIuMAM.jpg',
     'photo_807': 'https://pp.userapi.com/c638720/v638720358/3224d/mN0bMSCNPmw.jpg',
     'post_id': 3560,
     'text': '',
     'width': 750},
    'type': 'photo'}],
  'comments': {'can_post': 0, 'count': 0, 'groups_can_post': True},
  'date': 1493664166,
  'from_id': 23704358,
  'id': 3560,
  'likes': {'can_like': 1, 'can_publish': 1, 'count': 9, 'user_likes': 0},
  'owner_id': 23704358,
  'post_source': {'platform': 'instagram',
   'type': 'api',
   'url': 'https://www.instagram.com/p/BTgQBCqFgOO/'},
  'post_type': 'post',
  'reposts': {'count': 0, 'user_re

In [7]:
jposts[0].keys()

dict_keys(['attachments', 'comments', 'date', 'from_id', 'id', 'likes', 'owner_id', 'post_source', 'post_type', 'reposts', 'text', 'views'])

In [8]:
jposts[1013].keys()

dict_keys(['comments', 'copy_history', 'date', 'from_id', 'id', 'likes', 'owner_id', 'post_source', 'post_type', 'reposts', 'text'])

Трансформация json в dataframe реализовано в модуле vk_api
в двух методах - read_posts и read_users

In [9]:
post_df = vk_api.read_posts()
post_df.columns

Index(['owner_id', 'text', 'comments_count', 'likes_count', 'reposts_count',
       'date', 'post_type', 'views_count', 'copy_history_text',
       'copy_history_post_type', 'attachments_video_count',
       'attachments_photo_count', 'attachments_audio_count'],
      dtype='object')

In [10]:
user_df = vk_api.read_users()
print(user_df.columns)
user_df = user_df[['id','about', 'byear', 'books', 'activities', 'graduation',  'interests','movies', 'music', 'sex', 'relation' ]]

Index(['about', 'activities', 'bdate', 'books', 'byear', 'country_id',
       'country_title', 'education_form', 'education_status', 'faculty',
       'faculty_name', 'first_name', 'graduation', 'home_phone', 'home_town',
       'id', 'interests', 'last_name', 'mobile_phone', 'movies', 'music',
       'personal_alcohol', 'personal_inspired_by', 'personal_langs_0',
       'personal_langs_1', 'personal_langs_10', 'personal_langs_11',
       'personal_langs_12', 'personal_langs_13', 'personal_langs_14',
       'personal_langs_15', 'personal_langs_16', 'personal_langs_17',
       'personal_langs_18', 'personal_langs_19', 'personal_langs_2',
       'personal_langs_20', 'personal_langs_21', 'personal_langs_22',
       'personal_langs_23', 'personal_langs_24', 'personal_langs_25',
       'personal_langs_26', 'personal_langs_27', 'personal_langs_28',
       'personal_langs_29', 'personal_langs_3', 'personal_langs_4',
       'personal_langs_5', 'personal_langs_6', 'personal_langs_7',
       'pe

Итого определились с полями у пользователей и в постах

In [11]:
for col in post_df.columns:
    print(col)

owner_id
text
comments_count
likes_count
reposts_count
date
post_type
views_count
copy_history_text
copy_history_post_type
attachments_video_count
attachments_photo_count
attachments_audio_count


In [12]:
for col in user_df.columns:
    print(col)    

id
about
byear
books
activities
graduation
interests
movies
music
sex
relation


Заполним отсутствующий текст текстом из репоста, а если вообще нет текстат ни там ни там, заполним типом поста - есть варианты можно заполнить типа вложений

In [13]:
post_df.loc[post_df['text']=='','text'] = post_df['copy_history_text'].fillna(post_df['post_type'])

Удалим дубликаты

In [14]:
post_df = post_df.drop_duplicates()
post_df.count()

owner_id                   29929
text                       29929
comments_count             29929
likes_count                29929
reposts_count              29929
date                       29929
post_type                  29929
views_count                29929
copy_history_text          22236
copy_history_post_type     22236
attachments_video_count     6035
attachments_photo_count     6035
attachments_audio_count     6035
dtype: int64

Смержим с пользователями

In [15]:
post_df = post_df.merge(user_df, how="inner", left_on="owner_id",right_on='id')

Пермешаем посты  и для начала нам достаточно года рождения и текста, с них и начнем

In [16]:
#post_df
from sklearn.utils import shuffle
df = shuffle(post_df[['byear', 'text']])

In [18]:
df.sample(10)

,byear,text
7872,1991,Прежде чем сказать – послушай. \nПрежде чем на...
4636,1993,"Ти розказуй, розказуй, я завжди сиджу в наушни..."
28305,1986,)
9668,1989,😳
18704,1994,
24869,1993,😂😂😂
18002,1985,Маленькая прихожая. Арки сделаны из мозаики и ...
15415,1995,ГОРЯЧИЕ НОВИНКИ ЯНВАРЯ 2016 ГОДА \nСмотри перв...
13670,1983,
10506,1993,post


Разобьем на train и test

In [19]:
df_train = df[0:10000]
df_test = df[10001:20000]

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
x = count_vect.fit_transform(df_train['text']).todense()

In [21]:
x.shape

(10000, 66587)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(df_train['text'])

In [23]:
features.shape

(10000, 66587)

In [24]:
vectorizer.stop_words_

set()

In [26]:
vectorizer.get_feature_names()[10000:11110]

['вековых',
 'веком',
 'вектра',
 'веку',
 'вел',
 'вела',
 'велел',
 'велению',
 'веленью',
 'велес',
 'вели',
 'велик',
 'велика',
 'великанов',
 'великая',
 'великаяпобеда',
 'велике',
 'велики',
 'великие',
 'великий',
 'великим',
 'великих',
 'великобритании',
 'великобританию',
 'великобритания',
 'великобританский',
 'великовозрастный',
 'великого',
 'великодних',
 'великодні',
 'великое',
 'великой',
 'великолепие',
 'великолепия',
 'великолепна',
 'великолепная',
 'великолепно',
 'великолепной',
 'великолепную',
 'великолепны',
 'великолепные',
 'великолепный',
 'великолепных',
 'великом',
 'великому',
 'великую',
 'великі',
 'величайшие',
 'величайший',
 'величайших',
 'величезна',
 'величезного',
 'величества',
 'величественного',
 'величественный',
 'величественных',
 'величие',
 'величина',
 'величине',
 'величиной',
 'величину',
 'величины',
 'велк',
 'вело',
 'велогонщиков',
 'велогонщиком',
 'веломастерскую',
 'веломеханики',
 'велопробег',
 'велосипед',
 'велосипеда',


Уменьшим кол-во фитчей: стоп слова, max_df, min_df, max_features

In [44]:
from nltk.corpus import stopwords
print(' '.join(stopwords.words('russian')))
print(len(stopwords.words('russian')))

и в во не что он на я с со как а то все она так его но да ты к у же вы за бы по только ее мне было вот от меня еще нет о из ему теперь когда даже ну вдруг ли если уже или ни быть был него до вас нибудь опять уж вам ведь там потом себя ничего ей может они тут где есть надо ней для мы тебя их чем была сам чтоб без будто чего раз тоже себе под будет ж тогда кто этот того потому этого какой совсем ним здесь этом один почти мой тем чтобы нее сейчас были куда зачем всех никогда можно при наконец два об другой хоть после над больше тот через эти нас про всего них какая много разве три эту моя впрочем хорошо свою этой перед иногда лучше чуть том нельзя такой им более всегда конечно всю между
151


In [35]:
vectorizer = TfidfVectorizer( max_features=1000, min_df=0.005, max_df=0.99)
features = vectorizer.fit_transform(df_train['text'])
print(features.shape)

(10000, 630)


In [36]:
vectorizer.get_feature_names()

['00',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '150',
 '16',
 '17',
 '18',
 '180',
 '19',
 '20',
 '200',
 '2015',
 '2017',
 '21',
 '22',
 '23',
 '24',
 '25',
 '250',
 '26',
 '30',
 '300',
 '40',
 '400',
 '50',
 '500',
 '60',
 'com',
 'http',
 'https',
 'instagram',
 'minitest',
 'post',
 'ru',
 'st',
 'thepo',
 'vk',
 'www',
 'автор',
 'аккуратно',
 'аппетита',
 'без',
 'блюдо',
 'более',
 'больше',
 'большой',
 'будем',
 'будет',
 'буду',
 'будут',
 'бы',
 'бывает',
 'был',
 'была',
 'были',
 'было',
 'быстро',
 'быть',
 'важно',
 'вам',
 'вами',
 'вас',
 'ваш',
 'ваше',
 'вашей',
 'ваши',
 'вдруг',
 'ведь',
 'весь',
 'взбить',
 'взять',
 'виде',
 'видео',
 'вкус',
 'вкусно',
 'вкусу',
 'вместе',
 'вместо',
 'внимание',
 'внутри',
 'во',
 'вода',
 'воде',
 'водой',
 'воду',
 'воды',
 'возможно',
 'возможность',
 'вокруг',
 'вообще',
 'вопрос',
 'вот',
 'времени',
 'время',
 'все',
 'всегда',
 'всего',
 'всей',
 'всем',
 'всех',
 'всю',
 'всё',
 'второй',
 'вы',
 'выкладывае

фичи какие то мусорные, нужно применить стоп слова, стемминг или морфологию

In [105]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

print(list(map(lambda x: morph.parse(x)[0].normal_form, vectorizer.get_feature_names()))[200:210])

['куриный', 'кусочек', 'кусочек', 'легко', 'год', 'либо', 'лимон', 'лишь', 'ложка', 'ложка']


Лучше провести лемматизацию перед применением TfIdf

In [106]:
def preProcess(s):
    return morph.parse(s.lower())[0].normal_form

tfidf_vect = TfidfVectorizer(preprocessor=preProcess, stop_words=stopwords.words('russian'), max_features=1000, min_df=0.005, max_df=0.99)
features = tfidf_vect.fit_transform(df_train['text'])
print(features.shape)

(10000, 495)


In [107]:
tfidf_vect.get_feature_names()

['00',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '150',
 '16',
 '17',
 '18',
 '180',
 '19',
 '20',
 '200',
 '2015',
 '2017',
 '21',
 '22',
 '23',
 '24',
 '25',
 '250',
 '26',
 '30',
 '300',
 '40',
 '400',
 '50',
 '500',
 '60',
 'com',
 'http',
 'https',
 'instagram',
 'minitest',
 'post',
 'ru',
 'st',
 'thepo',
 'vk',
 'www',
 'автор',
 'аккуратно',
 'аппетита',
 'блюдо',
 'большой',
 'будем',
 'буду',
 'будут',
 'бывает',
 'быстро',
 'важно',
 'вами',
 'ваш',
 'ваше',
 'вашей',
 'ваши',
 'весь',
 'взбить',
 'взять',
 'виде',
 'видео',
 'вкус',
 'вкусно',
 'вкусу',
 'вместе',
 'вместо',
 'внимание',
 'внутри',
 'вода',
 'воде',
 'водой',
 'воду',
 'воды',
 'возможно',
 'возможность',
 'вокруг',
 'вообще',
 'вопрос',
 'времени',
 'время',
 'всей',
 'всем',
 'всё',
 'второй',
 'выкладываем',
 'выложить',
 'выпекать',
 'главное',
 'глаза',
 'говорит',
 'говорить',
 'говорят',
 'год',
 'года',
 'году',
 'голову',
 'город',
 'готов',
 'готовить',
 'готовности',
 'гр',
 'градусов',

Набор фитч выглядит относительно вменяемым, попробуем построить модель

In [108]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import time

def randomized_cv(model, param_grid, x_train, y_train):
    grid_search = RandomizedSearchCV(model, param_grid, cv=5, scoring='accuracy', n_iter=5)
    t_start = time.time()
    grid_search.fit(x_train, y_train)
    t_end = time.time()
    print('model {} best accuracy score is {}'.format(model.__class__.__name__, grid_search.best_score_))
    print('time for training is {} seconds'.format(t_end - t_start))
    return grid_search.best_estimator_

In [109]:
param_grid = {'alpha':[0.8, 0.09, 0.1, 1.1, 1.2]}
model = MultinomialNB()
X_train = features.todense()
y_train = df_train['byear']
best_model = randomized_cv(model, param_grid, X_train, y_train)
print(best_model)

model MultinomialNB best accuracy score is 0.1425
time for training is 2.7346320152282715 seconds
MultinomialNB(alpha=1.1, class_prior=None, fit_prior=True)


Модель не очень удачная посмотрим сколько у нас классов

In [110]:
y_train.describe()

count     10000
unique       30
top        1992
freq        999
Name: byear, dtype: object

Уменьшим кол-во классов

In [111]:
a = y_train.unique()
a.sort()
a

array(['1901', '1911', '1950', '1953', '1968', '1971', '1974', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002'], dtype=object)

In [112]:
bins = [1900, 1981, 1986, 1991, 1996, 2001, 2017]

In [113]:
import pandas as pd

y_less_class_train = pd.cut(y_train.astype(int),  bins=bins, labels=bins[1:])

In [114]:
param_grid = {'alpha':[0.8, 0.09, 0.1, 1.1, 1.2]}
model = MultinomialNB()
X_train = features.todense()
best_model = randomized_cv(model, param_grid, X_train, y_less_class_train)
print(best_model)

model MultinomialNB best accuracy score is 0.376
time for training is 1.295464038848877 seconds
MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)


In [116]:
X_test = tfidf_vect.transform(df_test['text']).todense()
y_pred = best_model.predict(X_test)
y_test = pd.cut(df_test['byear'].astype(int),  bins=bins, labels=bins[1:])

print(accuracy_score(y_test, y_pred))

0.3783378337833783


Попробуем SVM

In [118]:
from sklearn import svm
param_grid = {'C': [0.1, 0.5, 0.7, 1, 1.3, 1.5], 'kernel': ['linear', 'rbf']}
model = svm.SVC()
best_svc = randomized_cv(model, param_grid, X_train, y_less_class_train)
print(best_svc)

model SVC best accuracy score is 0.3817
time for training is 2461.4784836769104 seconds
SVC(C=0.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [119]:
y_pred = best_svc.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.37523752375237523


пока результаты удручающие будем думать дальше по возможости